In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from data_cleaner import clean_data

# altitude of home city 
# population of home city 

In [2]:
df15 = pd.read_csv('data/marathon_results_2015.csv')
df16 = pd.read_csv('data/marathon_results_2016.csv')
df17 = pd.read_csv('data/marathon_results_2017.csv')
df18 = pd.read_csv('data/marathon_results_2018.csv')
df18.rename(columns={'Blank': 'Unnamed'}, inplace = True)
print(df18.shape)
df18.head()

(24087, 26)


,Unnamed: 0,Unnamed: 0.1,Bib,Name,Age,M/F,City,State,Country,Citizen,...,25K,30K,35K,40K,Pace,Proj. Time,Official Time,Overall,Gender,Division
0,0,0,189,"Ash, Alan",26,M,Atmore,AL,USA,NaN,...,1:31:16,1:50:19,2:09:22,2:28:08,0:05:58,NaN,2:36:04,63,63,59
1,1,1,855,"Goodman, John Mark",38,M,Birmingham,AL,USA,NaN,...,1:34:33,1:53:49,2:13:27,2:33:06,0:06:11,NaN,2:41:52,187,186,173
2,2,2,1049,"Mcclung, Joshua",34,M,Birmingham,AL,USA,NaN,...,1:35:31,1:55:14,2:15:14,2:34:45,0:06:15,NaN,2:43:42,247,246,226
3,3,3,303,"Graham, James",26,M,Birmingham,AL,USA,NaN,...,1:33:09,1:52:29,2:13:49,2:37:17,0:06:23,NaN,2:47:07,385,377,346
4,4,4,5267,"Pierce, Nathaniel",31,M,Huntsville,AL,USA,NaN,...,1:40:23,2:00:42,2:21:22,2:41:26,0:06:30,NaN,2:50:19,608,593,536


In [86]:
import numpy as np
import pandas as pd


# def make_city_dict(path):
#     df = pd.read_csv(path)
    
#     city_dict = {}
#     for city, pop in zip(df['city'], df['population']):
#         city_dict[city] = pop
#     return city_dict    

# city_pop = make_city_dict('data/worldcities.csv')    


# def add_population(df):
#     pop_list = []
    
#     for city in df['City']:
#         try:
#             pop_list.append(city_pop[city])
#         except KeyError:
#             pop_list.append(np.nan)
#     df['home_pop'] = pop_list
#     return df


def clean_bib(bib):
    '''Strip the bib number and make an int'''
    return int(bib.strip('F'))


def seconds_converter(time):
    '''Takes a time in the string form '00:00:00' and converts it into integer
    of seconds.  Handles special '-' cases and converts them to NaN '''

    if time == '-':
        return np.nan

    time_split = [int(x) for x in (time.split(':'))]
    return time_split[0] * (60 ** 2) + time_split[1] * 60 + time_split[2]


def best_fit_slope(x, y):
    '''Calculate the best fit slope for all 5K splits.'''
    m = (((x.mean() * y.mean()) - (x * y).mean()) /
         (x.mean() ** 2 - (x ** 2).mean()))
    return round(m, 2)


def race_slope(row):
    '''Create a new column of the race progression slope.'''
    nK_splits = ['5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K']
    x = np.array([5, 10, 15, 20, 25, 30, 35, 40])
    y = np.array([row['5K']] + list(np.diff(row[nK_splits])))
    return best_fit_slope(x, y)


def race_slope_alternative(row):
    '''Create a new column of the race progression slope.'''
    
    if np.isnan(row['pace_slope']):
        half_splits = ['Half', 'finish_time']
        x = np.array([21.1, 42.2])
        y = np.array([row['Half']] + list(np.diff(row[half_splits])))
        return best_fit_slope(x, y) / 4
    return row['pace_slope']


def add_weather_data(df, year):
    '''Add the weather data for the corresponding year.'''

    year = int(year)
    median = df['finish_time'].median() / 3600

    pre_median_2015 = {'temp': 48, 'humidity': 0.71, 'wind': 17}
    post_median_2015 = {'temp': 44, 'humidity': 0.89, 'wind': 16}
    pre_median_2016 = {'temp': 64, 'humidity': 0.29, 'wind': 14}
    post_median_2016 = {'temp': 53, 'humidity': 0.55, 'wind': 14}
    pre_median_2017 = {'temp': 74, 'humidity': 0.29, 'wind': 21}
    post_median_2017 = {'temp': 73, 'humidity': 0.24, 'wind': 22}
    pre_median_2018 = {'temp': 45, 'humidity': 0.93, 'wind': 14}
    post_median_2018 = {'temp': 44, 'humidity': 1.00, 'wind': 17}

    new_cols = ['temp', 'humidity', 'wind']

    for col in new_cols:
        if year == 2015:
            df[col] = [pre_median_2015[col] if x < median else post_median_2015[col] for x in df['finish_time']]
        elif year == 2016:
            df[col] = [pre_median_2016[col] if x < median else post_median_2016[col] for x in df['finish_time']]
        elif year == 2017:
            df[col] = [pre_median_2017[col] if x < median else post_median_2017[col] for x in df['finish_time']]
        else:
            df[col] = [pre_median_2018[col] if x < median else post_median_2018[col] for x in df['finish_time']]

    return df

def overall_ratio(row, max_rank):
    return row['overall_rank'] / max_rank   

def gender_ratio(row, fmax, mmax):
    if row['Gender_F'] == 1:
        return row['gender_rank'] / fmax
    return row['gender_rank'] / mmax

def division_ratio(row, max1, max2, max3, max4, max5, max6, max7, max8, max9, max10):
    if row['Age'] < 40:
        return row['division_rank'] / max1
    if row['Age'] >= 40 and row['Age'] < 45:
        return row['division_rank'] / max2
    if row['Age'] >= 45 and row['Age'] < 50:
        return row['division_rank'] / max3
    if row['Age'] >= 50 and row['Age'] < 55:
        return row['division_rank'] / max4
    if row['Age'] >= 55 and row['Age'] < 60:
        return row['division_rank'] / max5
    if row['Age'] >= 60 and row['Age'] < 65:
        return row['division_rank'] / max6
    if row['Age'] >= 65 and row['Age'] < 70:
        return row['division_rank'] / max7
    if row['Age'] >= 70 and row['Age'] < 75:
        return row['division_rank'] / max8
    if row['Age'] >= 75 and row['Age'] < 80:
        return row['division_rank'] / max9
    if row['Age'] >= 80:
        return row['division_rank'] / max10
    
def rank_ratio(df):
    
    overall_max_rank = df['overall_rank'].max()
    df['overall_rank'] = df.apply(overall_ratio, max_rank=overall_max_rank, axis=1)

    female_max = df[df['Gender_F'] == 1]['gender_rank'].max()
    male_max = df[df['Gender_M'] == 1]['gender_rank'].max()
    df['gender_rank'] = df.apply(gender_ratio, axis=1, fmax=female_max, mmax=male_max)
    
    division1_max = df[df['Age'] < 40]['division_rank'].max()
    division2_max = df[(df['Age'] >= 40) & (df['Age'] < 45)]['division_rank'].max()
    division3_max = df[(df['Age'] >= 45) & (df['Age'] < 50)]['division_rank'].max()
    division4_max = df[(df['Age'] >= 50) & (df['Age'] < 55)]['division_rank'].max()
    division5_max = df[(df['Age'] >= 55) & (df['Age'] < 60)]['division_rank'].max()
    division6_max = df[(df['Age'] >= 60) & (df['Age'] < 65)]['division_rank'].max()
    division7_max = df[(df['Age'] >= 65) & (df['Age'] < 70)]['division_rank'].max()
    division8_max = df[(df['Age'] >= 70) & (df['Age'] < 75)]['division_rank'].max()
    division9_max = df[(df['Age'] >= 75) & (df['Age'] < 80)]['division_rank'].max()
    division10_max = df[df['Age'] >= 80]['division_rank'].max()
    df['division_rank'] = df.apply(division_ratio, axis=1, max1=division1_max, 
                                   max2=division2_max, max3=division3_max, 
                                   max4=division4_max, max5=division5_max, 
                                   max6=division6_max, max7=division7_max, 
                                   max8=division8_max, max9=division9_max, 
                                   max10=division10_max)
    return df
    


def clean_data(df, year):
    '''Takes in a DataFrame, drops unnecessary columns, and applies seconds_converter. '''

    # drop duplicates bib as they should be unique
    df.drop_duplicates(['Bib'], inplace=True)

    # clean the bib value convert to int
    df['Bib'] = df['Bib'].apply(clean_bib)

    # rename columns for clarity
    df.rename(columns={'Overall': 'overall_rank', 'Gender': 'gender_rank', 'Official Time': 'finish_time',
                       'Division': 'division_rank', 'M/F': 'Gender'}, inplace=True)

    # convert the time to seconds on these columns
    time_cols = ['5K', '10K', '15K', '20K', 'Half', '25K',
                 '30K', '35K', '40K', 'finish_time']
    for col in time_cols:
        df[col] = df[col].apply(seconds_converter)

    # create a new column pace_slope
    df['pace_slope'] = df.apply(race_slope, axis=1)
    df['pace_slope'] = df.apply(race_slope_alternative, axis=1)
    df['pace_slope'].fillna(df['pace_slope'].mean(), inplace=True)

    # add weather data
    df = add_weather_data(df, year)
    
    # add population of home city
#     df = add_population(df)

    # make gender binary
    df = pd.get_dummies(df, columns=['Gender'])

    # convert rankings to ratios
    df = rank_ratio(df)    

    # drop unnecessary columns
    df.drop(['Citizen', 'City', 'State', 'Country', '5K', '10K', '15K', '20K',
             '25K', '30K', '35K', '40K', 'Pace', 'Half', 'finish_time'], axis=1, inplace=True)
    df.drop(list(df.filter(regex='Unnamed')), axis=1, inplace=True)
    df.drop(list(df.filter(regex='Proj')), axis=1, inplace=True)

    return df
# , 'Half', 'finish_time'

In [87]:
# df15_clean = clean_data(df15.copy(), 2015)
# df16_clean = clean_data(df16.copy(), 2016)
# df17_clean = clean_data(df17.copy(), 2017)
df18_clean = clean_data(df18.copy(), 2018)
df18_clean.head()  

,Bib,Name,Age,overall_rank,gender_rank,division_rank,pace_slope,temp,humidity,wind,home_pop,Gender_F,Gender_M
0,189,"Ash, Alan",26,0.002439,0.004436,0.010190,1.84,44,1.0,17,6453.0,0,1
1,855,"Goodman, John Mark",38,0.007239,0.013096,0.029879,1.51,44,1.0,17,21007.0,0,1
2,1049,"Mcclung, Joshua",34,0.009562,0.017320,0.039033,1.98,44,1.0,17,21007.0,0,1
3,303,"Graham, James",26,0.014905,0.026544,0.059758,6.95,44,1.0,17,21007.0,0,1
4,5267,"Pierce, Nathaniel",31,0.023538,0.041752,0.092573,0.48,44,1.0,17,307319.0,0,1


In [89]:
df18.head()

,Unnamed: 0,Unnamed: 0.1,Bib,Name,Age,M/F,City,State,Country,Citizen,...,25K,30K,35K,40K,Pace,Proj. Time,Official Time,Overall,Gender,Division
0,0,0,189,"Ash, Alan",26,M,Atmore,AL,USA,NaN,...,1:31:16,1:50:19,2:09:22,2:28:08,0:05:58,NaN,2:36:04,63,63,59
1,1,1,855,"Goodman, John Mark",38,M,Birmingham,AL,USA,NaN,...,1:34:33,1:53:49,2:13:27,2:33:06,0:06:11,NaN,2:41:52,187,186,173
2,2,2,1049,"Mcclung, Joshua",34,M,Birmingham,AL,USA,NaN,...,1:35:31,1:55:14,2:15:14,2:34:45,0:06:15,NaN,2:43:42,247,246,226
3,3,3,303,"Graham, James",26,M,Birmingham,AL,USA,NaN,...,1:33:09,1:52:29,2:13:49,2:37:17,0:06:23,NaN,2:47:07,385,377,346
4,4,4,5267,"Pierce, Nathaniel",31,M,Huntsville,AL,USA,NaN,...,1:40:23,2:00:42,2:21:22,2:41:26,0:06:30,NaN,2:50:19,608,593,536


In [75]:
def race_slope_alternative(row):
    '''Create a new column of the race progression slope.'''
    
    if np.isnan(row['pace_slope']):
        half_splits = ['Half', 'finish_time']
        x = np.array([21.1, 42.2])
        y = np.array([row['Half']] + list(np.diff(row[half_splits])))
        return best_fit_slope(x, y) / 4
    return row['pace_slope']

race_slope_alternative(df18_clean.iloc[1])
# race_slope_alternative(test.iloc[0])

1.51

In [73]:
np.isnan(test.iloc[0]['pace_slope']) # is np.nan

True

In [71]:
test.iloc[38]['pace_slope'] is np.nan

False

In [22]:
df18_clean['pace_slope'].value_counts(dropna=False)

NaN       5135
 4.15       30
 6.00       28
 5.30       25
 3.60       24
 5.00       23
 2.70       23
 4.92       23
 5.15       23
 4.55       22
 3.15       22
 6.75       22
 5.62       22
 2.82       22
 5.25       22
 3.20       22
 4.20       21
 7.73       21
 6.80       21
 1.61       21
 2.60       21
 3.18       21
 3.06       21
 5.45       21
 7.08       20
 5.35       20
 8.65       20
 2.85       20
 4.35       20
 4.95       20
          ... 
 24.80       1
 34.79       1
-4.87        1
 36.75       1
 35.75       1
 25.82       1
-2.37        1
 38.37       1
 30.71       1
 20.50       1
 26.21       1
-2.61        1
 19.37       1
 24.11       1
 30.22       1
 48.60       1
 36.65       1
 26.03       1
 43.48       1
 30.65       1
 38.69       1
 36.94       1
 35.45       1
 28.74       1
 42.65       1
 30.35       1
 31.94       1
 28.28       1
 40.95       1
 27.19       1
Name: pace_slope, Length: 3438, dtype: int64

In [ ]:
plt.hist(df15['finish_time'], bins=20);  # poisson??? or just normal right skewed? log x values chack for normal

In [ ]:
df15_corr = df15[['Bib', 'Age', 'overall_rank', 'gender_rank', 
                  'division_rank', 'pace_slope']].corr()

fig, ax = plt.subplots(figsize=(15,4))
sns.heatmap(df15_corr, ax=ax, cmap="seismic", annot=True, 
            vmin=-1, vmax=1, linewidths=1, linecolor='ivory');

In [ ]:
corr_test = df15[['Bib', 'Age', 'overall_rank', 'gender_rank', 
                  'division_rank', 'pace_slope']].dropna().sample(1000)
sns.pairplot(corr_test, aspect=1.5);  # consider interaction with gender and gender_rank 

In [ ]:
legacy_runners = list(set(df15['Name']).intersection(df16['Name']).intersection(df17['Name']).intersection(df18['Name']))
len(legacy_runners)

In [ ]:
legacy_runners = (list(set(df15['Name'])
                       .intersection(df16['Name'])
                       .intersection(df17['Name'])
                       .intersection(df18['Name'])))  #1027 count

df_legacy15 = df15[df15['Name'].isin(legacy_runners)].sort_values(by='Name')
df_legacy16 = df16[df16['Name'].isin(legacy_runners)].sort_values(by='Name')
df_legacy17 = df17[df17['Name'].isin(legacy_runners)].sort_values(by='Name')
df_legacy18 = df18[df18['Name'].isin(legacy_runners)].sort_values(by='Name')
df_legacy16

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV


In [ ]:
# Create your feature matrix (X) and target vector (y)
mapping = ('finish_time ~ Bib + Age + overall_rank + gender_rank + division_rank + '
           'progression_slope + temp + dew_point + humidity + wind + gusts + Gender_M')
y, X = patsy.dmatrices(mapping, data=df_legacy15, return_type="dataframe")

# Create your model
model = sm.OLS(y, X)

# Fit your model to your training set
fit = model.fit()

# Print summary statistics of the model's performance
fit.summary()

In [ ]:
# Create your feature matrix (X) and target vector (y)
mapping = ('finish_time ~ Bib + Age + overall_rank + '
           'progression_slope + temp + dew_point + humidity + wind + Gender_M')
y, X = patsy.dmatrices(mapping, data=df_legacy15, return_type="dataframe")

# Create your model
model = sm.OLS(y, X)

# Fit your model to your training set
fit = model.fit()

# Print summary statistics of the model's performance
fit.summary()

In [ ]:
city_df = pd.read_csv('data/worldcities.csv')
city_df.tail()

In [ ]:
city_df.columns

In [ ]:
city_df[city_df['city'] == 'Atmore']

In [ ]:
city_df['admin_name'].value_counts(dropna=False)